In [329]:
### API CALL

In [ ]:
#############################################Retrieving data Starts here####################################################

In [366]:
import urllib.request
import urllib.parse
import json
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

In [ ]:
#Function to make API call and return the response
def api_call(url):
    with urllib.request.urlopen(url) as response:
        response = json.loads(response.read())
    return response

In [339]:
#Calling the API to retrieve records
url = "https://failteireland.azure-api.net/opendata-api/v1/activities"
previous_response = api_call(url)

In [340]:
#Taking the url for next API Call
df_prev = pd.DataFrame(previous_response['results'])

In [ ]:
#Calling the subsequent API to retrieve the records
next_response = previous_response
import time
for i in range(0,17):
    print("Outer loop {}".format(i))
    for count in range(0,8):
            print("Inner loop {}".format(count))
            url = next_response['nextPage']
            next_response = api_call(url)
            df_next = pd.DataFrame(next_response['results'])
            df_prev = pd.concat([df_prev, df_next], ignore_index=True)
    time.sleep(40) 

In [ ]:
#############################################Retrieving data Ends here####################################################

In [ ]:
#############################################Data Preprocessing Starts here####################################################

In [345]:
#Removing unwanted columns
df_prev.drop('image',
  axis='columns', inplace=True)

In [349]:
df = df_prev.copy()
region_list = []
#Changing the names of columns
df.rename(columns={'name' : 'restaurant_name', 
                   'url' : 'website', 
                   'tags' : 'activities'}, inplace = True)

In [352]:
#Splitting address object into Counties and Country
county_list = []
country_list = []
locality_list = []
for address in df['address']:
    county_list.append(address['addressRegion'])
    country_list.append(address['addressCountry'])
    locality_list.append(address['addressLocality'])
               
county_list =  pd.Series(county_list)
country_list =  pd.Series(country_list)
locality_list =  pd.Series(locality_list)       
df['locality_list'] = locality_list
df['addressRegion'] = county_list
df['addressCounty'] = country_list

In [357]:
#Splitting the activities
activities_array = []
for activities in df['activities']:
    for activity in activities:
        if activity not in activities_array:
            activities_array.append(activity)
            #Creating a column in df
            df[activity] = 0

In [359]:
#Put values in the activities column
row_count = 0
for activities in df['activities']:
    for activity in activities:
        if activity in activities_array:
            df[activity][row_count] = 1
    row_count = row_count+1

In [365]:
#Converting the type of the columns
df['Wild Atlantic Way'] = pd.to_numeric(df['Wild Atlantic Way'])

In [360]:
#Converting the Dataframe into CSV
df.to_csv("Activities.csv", header=True)

In [ ]:
################################################Data Preprocessing Ends#####################################################

In [363]:
###############################################Visualization Starts here####################################################

pandas.core.frame.DataFrame

In [ ]:
chart_cols = ['min_temperature','max_temperature']
filtered_data = df.loc['2021-01-01':'2021-01-31',chart_cols].reset_index()
filtered_data.columns = ['Date','Minimum','Maximum']
fig = px.line(filtered_data,
              x = 'Date',
              y = ['Minimum','Maximum'],
              labels = {'variable' : 'Series', 'value' : 'Temperature (&deg; C)'},
              title = 'Minimum and maximum temperatures at Dunsany, January 2021')
fig.show()

In [ ]:
chart_cols = ['min_temperature','max_temperature']
filtered_data = df.loc['2021-01-01':'2021-01-31',chart_cols].reset_index()
filtered_data.columns = ['Date','Minimum','Maximum']
fig = px.line(filtered_data,
              x = 'Date',
              y = ['Minimum','Maximum'],
              labels = {'variable' : 'Series', 'value' : 'Temperature (&deg; C)'},
              title = 'Minimum and maximum temperatures at Dunsany, January 2021')
fig.show()